In [7]:
import cv2
import numpy as np

win_name = "scanning"
img = cv2.imread("img/paper.jpg")
rows, cols = img.shape[:2]
draw = img.copy()
pts_cnt = 0
pts = np.zeros((4,2), dtype=np.float32)

ptsx = np.zeros((4,1), dtype=np.float32)
ptsy = np.zeros((4,1), dtype=np.float32)
ax = np.zeros((4,1), dtype=np.float32)
ay = np.zeros((4,1), dtype=np.float32)

compare=np.zeros((2,2), dtype=np.float32)

def onMouse(event, x, y, flags, param):  #마우스 이벤트 콜백 함수 구현 ---① 
    global  pts_cnt                     # 마우스로 찍은 좌표의 갯수 저장
    global whilei
    whilei=0
    if event == cv2.EVENT_LBUTTONDOWN:  
        cv2.circle(draw, (x,y), 10, (0,255,0), -1) # 좌표에 초록색 동그라미 표시
        print(x,y)
        cv2.imshow(win_name, draw)

        pts[pts_cnt] = [x,y]            # 마우스 좌표 저장
        ptsx[pts_cnt] = [x]
        ptsy[pts_cnt] = [y]
        
        pts_cnt+=1
        if pts_cnt == 4:                       # 좌표가 4개 수집됨
            #x y 좌표를 가장 작은순서부터 정렬한 행렬
            ax = np.sort(ptsx,axis=0)
            ay = np.sort(ptsy,axis=0)
            
            #y 좌표 중 작은 좌표 2개
            find_lowesty_x0,_ = np.where(pts==ay[0])
            find_lowesty_x1,_ = np.where(pts==ay[1])
            #print(find_lowesty_x1)
            compare_tl1 = pts[find_lowesty_x0[0]]
            compare_tl2 = pts[find_lowesty_x1[0]]

            
            #y 좌표 중 큰 좌표 2개
            find_highesty_x0,_ = np.where(pts==ay[2])
            find_highesty_x1,_ = np.where(pts==ay[3])
            
            compare_bl1 = pts[find_highesty_x0[0]]
            compare_bl2 = pts[find_highesty_x1[0]]
            
            if compare_tl1[0] < compare_tl2[0]:
                topLeft = pts[find_lowesty_x0[0]]
                topRight = pts[find_lowesty_x1[0]]
            else:
                topLeft = pts[find_lowesty_x1[0]]
                topRight = pts[find_lowesty_x0[0]]
                
            if compare_bl1[0] < compare_bl2[0]:
                bottomLeft = pts[find_highesty_x0[0]]
                bottomRight = pts[find_highesty_x1[0]]
            else:
                bottomLeft = pts[find_highesty_x1[0]]
                bottomRight = pts[find_highesty_x0[0]]

            
            
            # 좌표 4개 중 상하좌우 찾기 ---② 
            sm = pts.sum(axis=1)                 # 4쌍의 좌표 각각 x+y 계산
            diff = np.diff(pts, axis = 1)       # 4쌍의 좌표 각각 x-y 계산
            
            print('topLeft',topLeft)
            print('topRight',topRight)
            print('bottomLeft',bottomLeft)
            print('bottomRight',bottomRight)

            # 변환 전 4개 좌표 
            pts1 = np.float32([topLeft, topRight, bottomRight , bottomLeft])

            # 변환 후 영상에 사용할 서류의 폭과 높이 계산 ---③ 
            w1 = abs(bottomRight[0] - bottomLeft[0])    # 상단 좌우 좌표간의 거리
            w2 = abs(topRight[0] - topLeft[0])          # 하당 좌우 좌표간의 거리
            h1 = abs(topRight[1] - bottomRight[1])      # 우측 상하 좌표간의 거리
            h2 = abs(topLeft[1] - bottomLeft[1])        # 좌측 상하 좌표간의 거리
            width = max([w1, w2])                       # 두 좌우 거리간의 최대값이 서류의 폭
            height = max([h1, h2])                      # 두 상하 거리간의 최대값이 서류의 높이
            
            # 변환 후 4개 좌표
            pts2 = np.float32([[0,0], [width-1,0], 
                                [width-1,height-1], [0,height-1]])

            # 변환 행렬 계산 
            mtrx = cv2.getPerspectiveTransform(pts1, pts2)
            # 원근 변환 적용
            result = cv2.warpPerspective(img, mtrx, (width, height))
            cv2.imshow('scanned', result)
cv2.imshow(win_name, img)
cv2.setMouseCallback(win_name, onMouse)    # 마우스 콜백 함수를 GUI 윈도우에 등록 ---④
cv2.waitKey(0)
cv2.destroyAllWindows()

98 56
499 101
23 703
564 695
topLeft [98. 56.]
topRight [499. 101.]
bottomLeft [ 23. 703.]
bottomRight [564. 695.]
